In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt

C:\Users\user\AppData\Local\Temp\ipykernel_18888\4095300560.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### 현재 코드에서는, 20분 간격으로 레이블을 지정해주고, 한번에 그린 케이스

In [ ]:
# Define the path
path = 'D:\\MultiModal\\Face_z_score\\C_1\\'
save_path = 'D:\\MultiModal\\Face_z_score\\CC_plot\\'
if not os.path.exists(save_path):
    os.makedirs(save_path) 

# Define the lists
weeks = ['1W','2W','3W','4W']  # '2W', '3W', '4W'
steps = ['S1']  # 'S2' 'S3'
people = ['C1', 'C2', 'C3', 'C4']

fps = 25  # Frames per second

for week in weeks:
    for step in steps:
        
        temp = {}
        
        for person in people:
            file_path = os.path.join(path, f"Face_{week}_{person}_{step}.xlsx")
            if os.path.exists(file_path):
                df = pd.read_excel(file_path, index_col=0)
                
                df['box.size'] = df['box.height'] * df['box.width']
                # data sampling, 500번째에 찍기 
                #df = df.iloc[::500, :]
                df.index = pd.to_timedelta(df.index / fps, unit='s')  # Convert index to timedelta format based on fps
                df.index = df.index.astype(str).str.split(' ').str[-1]  # Convert timedelta to string and keep only the time part
                temp[person] = df

        # 플롯의 사이즈 조절
        plt.figure(figsize=(20, 10))

        for person in people:
            if person in temp:
                df = temp[person]
                plt.plot(df.index, df['box.size'], label=person)

        plt.xlabel('Time')
        plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
        
        
        # 시간을 매 10초씩 보여줄 수 있도록 세팅하는 부분.
        ax = plt.gca() # 현재 축 정보를 가져옴; 
        tick_labels = ax.get_xticklabels() # 현재 x축의 모든 레이블을 가져옴.
        tick_labels_to_keep = [tick_labels[i] for i in range(0, len(tick_labels), int(fps*20))] # 리스트를 하나 생성하고, 20초 간격으로 x축 정보를 설정함.
        ax.set_xticks(ax.get_xticks()[::int(fps*20)]) # x축 틱 정보를 20초 간격으로 설정함. 
        ax.set_xticklabels(tick_labels_to_keep) # 앞에서 설정한 x tick 정보를 넣어줌.
        
        # y축의 경우, box size로 유지.
        plt.ylabel('Box Size')
        plt.title(f'{week}_{step} graph')
        plt.legend()

        # Adjust layout to give more space for the x-axis labels
        plt.subplots_adjust(bottom=0.5)

        # Show the plot
        plt.show()
        
        # # Save the plot to the specified path
        # plot_filename = f"{week}_{step}_graph.png"
        # plot_filepath = os.path.join(save_path, plot_filename)
        # plt.savefig(plot_filepath)
        # plt.close()  # Close the plot to avoid display

### 지금 코드에서는 위의 그래프 시간 중, 첨예하게 잘라서 데이터가 잘 있는지 확인해볼 것.

In [ ]:
path = 'D:\\MultiModal\\Face_z_score\\C_1\\'
save_path = 'D:\\MultiModal\\Face_z_score\\1min_plot_C\\'
if not os.path.exists(save_path):
    os.makedirs(save_path) 

# Define the lists
weeks = ['1W','2W','3W','4W']  # '2W', '3W', '4W'
steps = ['S1', 'S2']  # 'S2', 'S3'
people = ['C1', 'C2', 'C3', 'C4']

fps = 25  
time_intervals = [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10),
                  (10,11), (11,12), (12,13), (13, 14), (14,15), (15,16), (16,17), (17,18), (18,19), (19,20)]  
# 원하는 시간 간격으로 나누어서, 그래프를 출력해볼 수 있는 부분. || 분 간격 

# time_intervals = [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5)]

for week in weeks:
    for step in steps:
        temp = {} # 빈 딕셔너리 생성, => 각 그룹별로 나누어 한번에 그릴 수 있도록 하기 위해서. 
        
        for person in people:
            file_path = os.path.join(path, f"Face_{week}_{person}_{step}.xlsx")
            if os.path.exists(file_path):
                df = pd.read_excel(file_path, index_col=0)
                
                df['box.size'] = df['box.height'] * df['box.width']
                df.index = pd.to_timedelta(df.index / fps, unit='s')  # Convert index to timedelta format based on fps
                df.index = df.index.astype(str).str.split(' ').str[-1]  # Convert timedelta to string and keep only the time part
                temp[person] = df

        # ~분부터 ~분까지 그릴 수 있도록 시간 값을 조절해줌.
        for start_minute, end_minute in time_intervals:
            # 플롯 사이즈 조절.
            plt.figure(figsize=(15, 10))
            start_time = pd.Timedelta(minutes=start_minute)
            end_time = pd.Timedelta(minutes=end_minute)

            for person in people:
                if person in temp:
                    df = temp[person]
                    df_interval = df[(pd.to_timedelta(df.index) >= start_time) & (pd.to_timedelta(df.index) < end_time)]
                    if not df_interval.empty:
                        plt.plot(df_interval.index, df_interval['box.size'], label=person)

            plt.xlabel('Time')
            plt.ylabel('Box Size')
            plt.title(f'{week}_{step} graph ({start_minute}min - {end_minute}min)')
            plt.legend()

            plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
            ax = plt.gca()
            tick_labels = ax.get_xticklabels()
            tick_labels_to_keep = [tick_labels[i] for i in range(0, len(tick_labels), int(fps*20))]
            ax.set_xticks(ax.get_xticks()[::int(fps*20)])
            ax.set_xticklabels(tick_labels_to_keep)

            # x축의 각 레이블 간의 사이즈 값을 조절해주기 위해, float 값을 줌. 
            plt.subplots_adjust(bottom=0.5)
            
            #plt.show()

            # Save the plot to the specified path
            plot_filename = f"{week}_{step}_graph_{start_minute}min_{end_minute}min.png"
            plot_filepath = os.path.join(save_path, plot_filename)
            plt.savefig(plot_filepath)
            plt.close

### 30초 간격으로 뽑고 싶을 때는, time interval 값을 어떻게 주어야 할까?

In [ ]:
path = 'D:\\MultiModal\\Face_z_score\\A_1\\'
save_path = 'D:\\MultiModal\\Face_z_score\\30s_plot_A\\'
if not os.path.exists(save_path):
    os.makedirs(save_path) 

# Define the lists
weeks = ['1W','2W','3W','4W']  # '2W', '3W', '4W'
steps = ['S1', 'S2']  # 'S2', 'S3'
people = ['A1', 'A2', 'A3', 'A4']

fps = 25 # fps 영상 
total_seconds = 20 * 60  # 전체 시간 : 20분, 시간 간격은 30초로 주고 싶기애. Total duration in seconds (20 minutes)
interval_seconds = 30  # 30초 단위로 간격을 주고자 선언.

time_intervals = [(i, i + interval_seconds) for i in range(0, total_seconds, interval_seconds)]
# i 는 0부터 시작하게 되고, 30초 마다 더해져서 값이 나오게 됨.

for week in weeks:
    for step in steps:
        
        temp = {}
        
        for person in people:
            file_path = os.path.join(path, f"Face_{week}_{person}_{step}.xlsx")
            if os.path.exists(file_path):
                df = pd.read_excel(file_path, index_col=0)
                
                df['box.size'] = df['box.height'] * df['box.width']
                df.index = pd.to_timedelta(df.index / fps, unit='s')  # Convert index to timedelta format based on fps
                df.index = df.index.astype(str).str.split(' ').str[-1]  # Convert timedelta to string and keep only the time part
                temp[person] = df

        for start_second, end_second in time_intervals:
            plt.figure(figsize=(15, 10))
            start_time = pd.Timedelta(seconds=start_second)
            end_time = pd.Timedelta(seconds=end_second)

            for person in people:
                if person in temp:
                    df = temp[person]
                    df_interval = df[(pd.to_timedelta(df.index) >= start_time) & (pd.to_timedelta(df.index) < end_time)]
                    if not df_interval.empty:
                        plt.plot(df_interval.index, df_interval['box.size'], label=person)

            plt.xlabel('Time')
            plt.ylabel('Box Size')
            plt.title(f'{week}_{step} graph ({start_second//60}min {start_second%60}s - {end_second//60}min {end_second%60}s)')
            plt.legend()

            plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
            ax = plt.gca()
            tick_labels = ax.get_xticklabels()
            tick_labels_to_keep = [tick_labels[i] for i in range(0, len(tick_labels), int(fps*20))]
            ax.set_xticks(ax.get_xticks()[::int(fps*20)])
            ax.set_xticklabels(tick_labels_to_keep)

            # Adjust layout to give more space for the x-axis labels
            plt.subplots_adjust(bottom=0.5)

            # Show the plot
            #plt.show()

            # Save the plot to the specified path
            plot_filename = f"{week}_{step}_graph_{start_second//60}min_{start_second%60}s_{end_second//60}min_{end_second%60}s.png"
            plot_filepath = os.path.join(save_path, plot_filename)
            plt.savefig(plot_filepath)
            plt.close() 

### 시작 시간과 끝나는 시간에 각각 빨간색(=시작 시간), 파란색(=끝나는 시간)으로 표시해보자

In [4]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Define the path
path = 'D:\\MultiModal\\Face_z_score\\C_1\\'
save_path = 'D:\\MultiModal\\Face_z_score\\result_1min_plot_C\\'
if not os.path.exists(save_path):
    os.makedirs(save_path) 

# Define the lists
weeks = ['1W','2W','3W','4W']  # '2W', '3W', '4W'
steps = ['S1']  # 'S2', 'S3'
people = ['C1', 'C2', 'C3', 'C4']

fps = 25  
time_intervals = [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10),
                  (10,11), (11,12), (12,13), (13, 14), (14,15), (15,16), (16,17), (17,18), (18,19), (19,20)]  
# 원하는 시간 간격으로 나누어서, 그래프를 출력해볼 수 있는 부분. || 분 간격 


# A 그룹 시작 시간.
# start_times = [
#     pd.Timedelta(seconds=3), pd.Timedelta(minutes=3, seconds=30),
#     pd.Timedelta(minutes=4, seconds=30), pd.Timedelta(minutes=6, seconds=9),
#     pd.Timedelta(minutes=7, seconds=19), pd.Timedelta(minutes=11, seconds=58),
#     pd.Timedelta(minutes=12, seconds=55), pd.Timedelta(minutes=13),
#     pd.Timedelta(minutes=15, seconds=28)
# ]

# A 그룹 끝나는 시간.
# end_times = [
#     pd.Timedelta(seconds=6), pd.Timedelta(minutes=3, seconds=35),
#     pd.Timedelta(minutes=4, seconds=50), pd.Timedelta(minutes=6, seconds=22),
#     pd.Timedelta(minutes=7, seconds=30), pd.Timedelta(minutes=12),
#     pd.Timedelta(minutes=13), pd.Timedelta(minutes=13, seconds=9),
#     pd.Timedelta(minutes=15, seconds=30)
# ]


# C 그룹 시작 시간.
start_times = [
    pd.Timedelta(minutes=16, seconds=32),
    pd.Timedelta(minutes=17, seconds=37), pd.Timedelta(minutes=18, seconds=7)
]

# C 그룹 끝나는 시간.
end_times = [
    pd.Timedelta(minutes=16, seconds=33),
    pd.Timedelta(minutes=17, seconds=38), pd.Timedelta(minutes=18, seconds=10)
]


# 선을 그릴 때, 선의 간격을 조절해주기 위해서 offset 값 설정.
offset = pd.Timedelta(seconds=0.5)


for week in weeks:
    for step in steps:
        temp = {} # 빈 딕셔너리 생성, => 각 그룹별로 나누어 한번에 그릴 수 있도록 하기 위해서. 
        
        for person in people:
            file_path = os.path.join(path, f"Face_{week}_{person}_{step}.xlsx")
            if os.path.exists(file_path):
                df = pd.read_excel(file_path, index_col=0)
                
                df['box.size'] = df['box.height'] * df['box.width']
                df.index = pd.to_timedelta(df.index / fps, unit='s')  # Convert index to timedelta format based on fps
                df.index = df.index.astype(str).str.split(' ').str[-1]  # Convert timedelta to string and keep only the time part
                temp[person] = df

        # ~분부터 ~분까지 그릴 수 있도록 시간 값을 조절해줌.
        for start_minute, end_minute in time_intervals:
            # 플롯 사이즈 조절.
            plt.figure(figsize=(15, 10))
            start_time = pd.Timedelta(minutes=start_minute)
            end_time = pd.Timedelta(minutes=end_minute)

            for person in people:
                if person in temp:
                    df = temp[person]
                    df_interval = df[(pd.to_timedelta(df.index) >= start_time) & (pd.to_timedelta(df.index) < end_time)]
                    if not df_interval.empty:
                        plt.plot(df_interval.index, df_interval['box.size'], label=person)

            plt.xlabel('Time')
            plt.ylabel('Box Size')
            plt.title(f'{week}_{step} graph ({start_minute}min - {end_minute}min)')
            plt.legend()

            plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
            ax = plt.gca()
            tick_labels = ax.get_xticklabels()
            tick_labels_to_keep = [tick_labels[i] for i in range(0, len(tick_labels), int(fps*20))]
            ax.set_xticks(ax.get_xticks()[::int(fps*20)])
            ax.set_xticklabels(tick_labels_to_keep)

            # x축의 각 레이블 간의 사이즈 값을 조절해주기 위해, float 값을 줌. (bottom 부분의 간격 값을 조정해주기 위해서 넣어줌.)
            plt.subplots_adjust(bottom=0.5)

            # Highlight the specific start and end times
            for highlight_time in start_times:
                adjust_time = highlight_time + offset
                if start_time <= adjust_time < end_time:
                    plt.axvline(x=str(adjust_time), color='red', linewidth=2, linestyle='--', label='Start Time')
            for highlight_time in end_times:
                adjust_time = highlight_time - offset
                if start_time <= adjust_time < end_time:
                    plt.axvline(x=str(adjust_time), color='blue', linewidth=2, linestyle='--', label='End Time')


            # 그래프 시각화
            #plt.show()

            # Save the plot to the specified path
            # plot_filename = f"{week}_{step}_graph_{start_minute}min_{end_minute}min.png"
            # plot_filepath = os.path.join(save_path, plot_filename)
            # plt.savefig(plot_filepath)
            # plt.close()
